<a href="https://colab.research.google.com/github/abrichr/visual-contact-tracing/blob/master/Visual_Contact_Tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install detectron2

In [1]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.4 torchvision==0.5 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.4 in /usr/local/lib/python3.6/dist-packages (1.4.0)
Requirement already up-to-date: torchvision==0.5 in /usr/local/lib/python3.6/dist-packages (0.5.0)
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-n9722vpq
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-n9722vpq
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=275268 sha256=6f1b225791ac8961eb973265ef37b1b4d51f9cbb7c73e2053d24fa6fa765a17e
  Stored in directory: /tmp/pip-ephem-wheel-cache-j9opg8a5/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
1.4.0 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundati

In [2]:
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html


In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

Upload a video

In [4]:
'''
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
'''

'\nfrom google.colab import files\n\nuploaded = files.upload()\n\nfor fn in uploaded.keys():\n  print(\'User uploaded file "{name}" with length {length} bytes\'.format(\n      name=fn, length=len(uploaded[fn])))\n'

Or load from Google Drive

In [5]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

file_id = '0Bzf1l8WmTwu0eUluQ2h1NWZQRjQ'
video_filename = 'salsa_cpp_cam4.avi'
download_file_from_google_drive(file_id, video_filename)
with open(video_filename, 'rb') as f:
  data = f.read()
  print('downloaded', len(data), 'bytes to', video_filename)

downloaded 150146578 bytes to salsa_cpp_cam4.avi


Re-encode

In [6]:
import subprocess as sp

import imageio
                                                                    
ffmpeg = imageio.plugins.ffmpeg                                                
try:                                                                           
    ffmpeg.download()                                                          
except:                                                                        
    pass                                                                       
ffmpeg_exe = ffmpeg.get_exe()

video_path = video_filename 
start_time_seconds = 1
duration_seconds = 5
video_filename_reenc = video_filename + '-reenc.avi'
cmd_parts = [
  ffmpeg_exe,
  '-i', video_path,
  '-vcodec', 'h264',
  '-acodec', 'aac',
  '-strict',
  '-2',
  '-ss', str(start_time_seconds),
  '-t', str(duration_seconds),
  '-y',
  video_filename_reenc
]
print('Running cmd:\n', ' '.join(cmd_parts))                                
result = sp.run(                                                            
    cmd_parts, stdout=sp.PIPE, stderr=sp.STDOUT, universal_newlines=True    
)                                                                           
print('Result:\n', result.stdout)                                           

with open(video_filename_reenc, 'rb') as f:
  data = f.read()
  print('wrote', len(data), 'bytes')

Running cmd:
 /root/.imageio/ffmpeg/ffmpeg-linux64-v3.3.1 -i salsa_cpp_cam4.avi -vcodec h264 -acodec aac -strict -2 -ss 1 -t 5 -y salsa_cpp_cam4.avi-reenc.avi
Result:
 ffmpeg version N-86111-ga441aa90e8-static http://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 5.4.1 (Debian 5.4.1-8) 20170304
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc-5 --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gray --enable-libass --enable-libfreetype --enable-libfribidi --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-librtmp --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libzimg
  libavutil      55. 63.100 / 55. 63.1

In [7]:
!git clone https://github.com/facebookresearch/detectron2

fatal: destination path 'detectron2' already exists and is not an empty directory.


In [12]:
import argparse
import glob
import multiprocessing as mp
import os
import time
import cv2
import tqdm

from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.video_visualizer import VideoVisualizer


# panoptic segmentation
DEFAULT_CONFIG = 'detectron2/configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml'
DEFAULT_OPTS = ['MODEL.WEIGHTS', 'detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl']
DEFAULT_CONF_THRESH = 0.6

# keypoint detection
DEFAULT_CONFIG = 'detectron2/configs/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml'
DEFAULT_CONF_THRESH = 0.7
DEFAULT_OPTS = [
  'MODEL.WEIGHTS',
  model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
]

default_args = [DEFAULT_CONFIG, DEFAULT_OPTS, DEFAULT_CONF_THRESH]

def setup_cfg(config=DEFAULT_CONFIG, opts=DEFAULT_OPTS, conf_thresh=DEFAULT_CONF_THRESH):
    # load config from file and arguments
    cfg = get_cfg()
    cfg.merge_from_file(config)
    cfg.merge_from_list(opts)
    # Set score_threshold for builtin models
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = conf_thresh
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = conf_thresh
    cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = conf_thresh
    cfg.freeze()
    return cfg

mp.set_start_method("spawn", force=True)
setup_logger(name="fvcore")
logger = setup_logger()
logger.info("Arguments: " + str(default_args))

cfg = setup_cfg()
predictor = DefaultPredictor(cfg)

video_input = video_filename_reenc
print('video_input:', video_input)
output = f'{video_filename_reenc}-out.mp4'
print('output:', output)

video = cv2.VideoCapture(video_input)
print('video:', video)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
basename = os.path.basename(video_input)

output_fname = './' + output
print('output_fname:', output_fname)
#assert not os.path.isfile(output_fname), output_fname
output_file = cv2.VideoWriter(
    filename=output_fname,
    # some installation of opencv may not support x264 (due to its license),
    # you can try other format (e.g. MPEG)
    #fourcc=cv2.VideoWriter_fourcc(*"x264"),
    #fourcc=cv2.VideoWriter_fourcc(*"MPEG"),
    fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
    fps=float(frames_per_second),
    frameSize=(width, height),
    isColor=True,
)
print('output_file:', output_file)
assert os.path.isfile(video_input)

metadata = MetadataCatalog.get(
    cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) else "__unused"
)
cpu_device = torch.device("cpu")
instance_mode = ColorMode.IMAGE
video_visualizer = VideoVisualizer(metadata, instance_mode)

def process_predictions(frame, predictions):
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    if "panoptic_seg" in predictions:
        panoptic_seg, segments_info = predictions["panoptic_seg"]
        vis_frame = video_visualizer.draw_panoptic_seg_predictions(
            frame, panoptic_seg.to(cpu_device), segments_info
        )
    elif "instances" in predictions:
        predictions = predictions["instances"].to(cpu_device)
        vis_frame = video_visualizer.draw_instance_predictions(frame, predictions)
    elif "sem_seg" in predictions:
        vis_frame = video_visualizer.draw_sem_seg(
            frame, predictions["sem_seg"].argmax(dim=0).to(cpu_device)
        )

    # Converts Matplotlib RGB format to OpenCV BGR format
    vis_frame = cv2.cvtColor(vis_frame.get_image(), cv2.COLOR_RGB2BGR)
    return vis_frame

def _frame_from_video(video):
    while video.isOpened():
        success, frame = video.read()
        if success:
            yield frame
        else:
            break

frame_gen = _frame_from_video(video)
from google.colab.patches import cv2_imshow
for frame in tqdm.tqdm(frame_gen, total=num_frames):
    predictions = predictor(frame)
    # XXX TODO: track positions across frames
    vis_frame = process_predictions(frame, predictions)
    #cv2_imshow(vis_frame)
    output_file.write(vis_frame)


video.release()
output_file.release()
print('Done.')

with open(output_fname, 'rb') as f:
  print('wrote', len(f.read()), 'bytes')


[05/02 03:29:02 detectron2]: Arguments: ['detectron2/configs/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml', ['MODEL.WEIGHTS', 'https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl'], 0.7]
[05/02 03:29:03 fvcore.common.checkpoint]: Loading checkpoint from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x/137849621/model_final_a6e10b.pkl
[05/02 03:29:03 fvcore.common.checkpoint]: Reading a file from 'Detectron2 Model Zoo'


  0%|          | 0/75 [00:00<?, ?it/s]

video_input: salsa_cpp_cam4.avi-reenc.avi
output: salsa_cpp_cam4.avi-reenc.avi-out.mp4
video: <VideoCapture 0x7fe8d5da0af0>
output_fname: ./salsa_cpp_cam4.avi-reenc.avi-out.mp4
output_file: <VideoWriter 0x7fe8d5da0210>


100%|██████████| 75/75 [00:38<00:00,  1.93it/s]

Done.
wrote 4849365 bytes


In [0]:
# Download the results
from google.colab import files
files.download(output)

Limitations
- 2D results in false positives (due to lack of depth information) and false negatives (due to occlusion)